In [1]:
import sys
sys.path.append('../')

import time
import random
import numpy as np
import torch
import torch.tensor as t
import torch.nn as nn
from random import randint
from utils.decorators import timer

In [ ]:
batch = 8
height = 50
width = 50

In [ ]:
x = torch.rand(batch, 3, height, width)
x = nn.Softmax(dim=1)(x)
print(x.shape)

In [ ]:
for i in range(10):
    b, h, w = randint(0, batch-1), randint(0, height-1), randint(0, width-1)
    print(x[b, :, h, w].sum().item())

In [ ]:
x_new = x.permute(0, 2, 3, 1).contiguous().view(-1, 3)
# x_new = x_new
print(x_new.shape)

In [ ]:
n = x_new.shape[1]
for i in range(100):
    idx = randint(0, n-1)
    print(x_new[idx, :].sum().item())

In [7]:
# Mask length

x = torch.rand(200,)>0.5
mask_0 = x==0
mask_non0 = x!=0
print(len(x), len(mask_0), len(mask_non0))

mask_0 = torch.nonzero(mask_0)
mask_non0 = torch.nonzero(mask_non0)
print(len(x), len(mask_0), len(mask_non0))


200 200 200
200 106 94


In [28]:
# Time

@timer
def f(x, factor=1):
    mask_0 = torch.nonzero(x==0)
    mask_non0 = torch.nonzero(x!=0)
    n_neg, n_pos = len(mask_0), len(mask_non0)
    
    t1 = time.time()
    n_neg_need = int(factor*n_pos)
    # mask_mask_0 = torch.randint(len(mask_0), (n_neg_need,))       # 0.01 sec
    # mask_mask_0 = random.sample(range(len(mask_0)), k=n_neg_need) # 0.9 sec
    mask_mask_0 = torch.randperm(len(mask_0))[:n_neg_need]        # 0.11 sec
    print(time.time()-t1)
    print(len(mask_mask_0), len(np.unique(mask_mask_0)))    
    
    mask_0 = mask_0[mask_mask_0]
    mask = (x!=0)
    mask[mask_0] = True
    return mask

x = torch.rand(2048*2048,)>0.8
mask = f(x)
x = x[mask]

n_pos = np.count_nonzero(x)
n_neg = len(x)-n_pos
print(n_pos, n_neg)

0.1169424057006836
838589 838589
Time : 0.32019925117492676 seconds
838589 838589
